# <span style="color:purple">Clustering: </span><span>Finding Restaurant Clusters for Retail Site Selection</span>

![clustering](img/restaurants_clustering_v2.gif "Density-based Clustering")


## An Example of Spatial Analysis in ArcGIS Using Density-based Clustering 

Prospecting a successful retail location often requires a thorough understanding of where people shop, dine, and linger for entertainment in a study area. One useful way to start making sense of the vast quantities of retail business information and start finding the locations that matter is to use spatial clustering to find locations with a dense retail presence. 

This example starts with retail business data in Pittsburgh, PA and leverages a couple of Spatial Statistics tools in ArcGIS ([Density-based Clustering](http://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/densitybasedclustering.htm), [Directional Distribution (Standard Deviational Ellipse)](http://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/directional-distribution.htm)) to separate the dense retail  data into defined cluster locations which can then be the target of further analysis and potential decisions for your organization. 

## Steps

This is a basic example that will cover the following steps:

1) Retrieve Pittsburgh restaurants data.


2) Explore the data through tables, charts, and maps.


3) Perform spatial analysis:

    - Density-based Clustering
    - Directional Distribution (Standard Deviational Ellipse)


4) Present Results.

### Preliminary Setup

#### Import the needed modules, including ArcPy, the ArcGIS API for Python, and other useful modules

In [134]:
import arcpy
arcpy.env.overwriteOutput = True
import arcgis
from arcgis import features
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geoenrichment import *
import os

In [135]:
# Imports for plotting in Bokeh
import numpy as np
import bokeh
from bokeh.io import output_notebook, output_file, show
from bokeh.plotting import figure
from bokeh.models import Legend, Range1d
from bokeh.embed import file_html
from bokeh.resources import CDN
# Set bokeh to output plots in the notebook
output_notebook()

Loading BokehJS ...

In [136]:
gis = arcgis.gis.GIS("home", verify_cert=False)

In [137]:
# neent_gis = arcgis.gis.GIS("http://neenterprise.esri.com/portal/", username="anieto")

# <span style="color:purple">1) Retrieve Pittsburgh restaurants data</span>

Using Business Analyst data, we published an item to the ArcGIS Enterprise. Let's find this item using the "Add" toolbar and load it into our notebook as an item variable.

In [138]:
# Item Added From Toolbar
# Title: Restaurant Points v1 | Type: Feature Service | Owner: anieto001
restaurants_item = gis.content.get("bb8eefe085fe4ab08e920bc67e5bde6a")
restaurants_item

<Item title:"Restaurant Points v1" type:Feature Layer Collection owner:anieto001>

The spatially-enabled dataframe helps us explore the data in a tabular format by leveraging the Pandas library.

In [139]:
restaurants_sdf = pd.DataFrame.spatial.from_layer(restaurants_item.layers[0])
restaurants_sdf.head()

,SHAPE,city,coname,empnum,frncod,hdbrch,iscode,loc_name,locnum,naics,...,score,sic,source,sqftcode,state,state_name,status,street,zip,zip4
0,"{""x"": -8919964.005599998, ""y"": 4867758.9117000...",MARIANNA,RUDNICK'S,6.0,,,3,StreetAddress,402036338,72251117,...,100.0000,581208,INFOGROUP,A,PA,Pennsylvania,M,TEN MILE RD,15345,1080
1,"{""x"": -8905258.700800002, ""y"": 4865826.0264, ""...",FREDERICKTOWN,RIVERSIDE INN,5.0,,,,StreetAddress,387410335,72251117,...,90.0938,581208,INFOGROUP,A,PA,Pennsylvania,M,FRONT ST,15333,
2,"{""x"": -8905314.3606, ""y"": 4866727.023800001, ""...",FREDERICKTOWN,RIVIERA RESTAURANT LOUNGE,5.0,,,,StreetAddress,831505334,72251117,...,100.0000,581208,INFOGROUP,A,PA,Pennsylvania,M,FRONT ST,15333,
3,"{""x"": -8904590.783900002, ""y"": 4868224.0231000...",BETHLEHEM,KUKI CHINESE RESTAURANT,3.0,,,C,PointAddress,907890941,72251117,...,86.8281,581208,INFOGROUP,A,PA,Pennsylvania,M,E 3RD ST,18015,1306
4,"{""x"": -8896241.822100002, ""y"": 4844052.5718000...",MASONTOWN,DOLFI'S,10.0,,,,StreetAddress,309912996,72251117,...,100.0000,581208,INFOGROUP,B,PA,Pennsylvania,M,RIVER AVE,15461,1551


# <span style="color:purple">2. Explore the data through tables, charts, and maps.</span>

In [140]:
# Get the amount of records and columns in our data
print("Amount of restaurants: {0}\nAmount of attributes: {1}".format(restaurants_sdf.shape[0], restaurants_sdf.shape[1]))

Amount of restaurants: 5026
Amount of attributes: 22


In [141]:
# Get summary statistics on numeric fields
restaurants_sdf.describe()

,empnum,objectid,salesvol,score
count,5026.000000,5026.000000,5026.000000,5026.000000
mean,17.216076,2513.500000,802.645444,99.608847
std,24.190230,1451.025557,1130.667265,1.855258
min,1.000000,1.000000,0.000000,85.703100
25%,6.000000,1257.250000,280.000000,100.000000
50%,8.000000,2513.500000,374.000000,100.000000
75%,19.000000,3769.750000,888.000000,100.000000
max,300.000000,5026.000000,14035.000000,100.000000


In [142]:
# Get a basic map of the restaurant locations as points
restaurants_map = gis.map("Pittsburgh")
restaurants_sdf.spatial.plot(map_widget=restaurants_map)
restaurants_map

MapView(layout=Layout(height='400px', width='100%'))

In [143]:
# Create a histogram of salesvol values
hist, edges = np.histogram(restaurants_sdf['salesvol'],
                          bins=32)

# Put the information in a dataframe
salesvol_df = pd.DataFrame({'arr_salesvol': hist,
                            'left': edges[:-1],
                            'right': edges[1:]})

# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram of Sales Volume',
           y_axis_label = 'Restaurants',
           x_axis_label = 'Sales volume in thousands of dollars')

# Add a quad glyph
p.quad(bottom=0, top=salesvol_df['arr_salesvol'], 
       left=salesvol_df['left'], right=salesvol_df['right'],
       line_color='white')

# Show the plot
show(p)

See https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F00%20-%20Introduction%20and%20Setup.ipynb for a guide to Bokeh's plotting capabilities

## What are the top five restaurants by sales volume?

In [144]:
# Get the restaurants with the highest sales revenue by sorting on the 'SALESVOL' column
top_sales_sdf = restaurants_sdf.sort_values(by=['salesvol'], ascending=False).reset_index()
top_five_sdf = top_sales_sdf.head()

print("Top 5 Restaurants in Pittsburgh by Sales Volume: ")
top_five_sdf[['coname', 'salesvol', 'street', 'city', 'state', 'state_name', 'zip']]

Top 5 Restaurants in Pittsburgh by Sales Volume: 


,coname,salesvol,street,city,state,state_name,zip
0,CHEESECAKE FACTORY,14035.0,ROSS PARK MALL DR,PITTSBURGH,PA,Pennsylvania,15237
1,EAT'N PARK,12865.0,PARK MANOR DR,PITTSBURGH,PA,Pennsylvania,15205
2,CHEESECAKE FACTORY,12631.0,S 27TH ST,PITTSBURGH,PA,Pennsylvania,15203
3,M I FRIDAY,11695.0,PERRY HWY,PITTSBURGH,PA,Pennsylvania,15229
4,NANA,11695.0,WATERFRONT DR,PITTSBURGH,PA,Pennsylvania,15222


In [145]:
# Create a basic chart of the top five restaurants by sales volume
label = top_five_sdf['street'].tolist()
value = top_five_sdf['salesvol'].tolist()

p = figure(x_range=label,
           title = 'Top Five Restaurants by Sales Volume',
           x_axis_label = 'Restaurant Address',
           y_axis_label = 'Sales volume in thousands of dollars')
p.xaxis.major_label_orientation = np.pi/4

p.vbar(x=label, top=value, width=0.5)

show(p)

In [146]:
# Plot the highest five selling restaurants
top_five_restaurants_map = gis.map("Pittsburgh")
top_five_sdf.spatial.plot(map_widget=top_five_restaurants_map)
top_five_restaurants_map

/opt/conda/lib/python3.6/site-packages/arcgis/features/geo/_accessor.py:865: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


MapView(layout=Layout(height='400px', width='100%'))

## What are the top five zip codes by sales volume?

In [147]:
# Determine the top five zip codes by average restaurant sales volume

# Use groupby object to make calculations by Zip Code
zip_groupby = restaurants_sdf.groupby(restaurants_sdf['zip'])
# Determine means for numeric columns by zip
zip_mean_sales_df = zip_groupby.mean().sort_values(by=['salesvol'], ascending=False)
# Calculate Zip ranks by average sales volume
zip_mean_sales_df['Zip Ave. Sales Rank'] = zip_mean_sales_df['salesvol'].rank(ascending=False)
# Drop the unneeded 'SCORE' field
del zip_mean_sales_df['score']

zip_mean_sales_df.head()

,empnum,objectid,salesvol,Zip Ave. Sales Rank
zip,,,,
15231,39.818182,4293.818182,1862.272727,1.0
15275,38.521739,3861.695652,1801.652174,2.0
15482,35.000000,950.000000,1637.000000,3.0
15084,32.605263,1093.789474,1524.842105,4.0
15205,30.424779,3716.203540,1422.796460,5.0


In [148]:
# Create a basic chart of the top five zip codes by sales volume
label = list(zip_mean_sales_df.head().index.values)
value = list(zip_mean_sales_df['salesvol'].head())

p = figure(x_range=label,
           title = 'Top Five Zip Codes by Mean Sales Volume',
           x_axis_label = 'Zip Code',
           y_axis_label = 'Average Sales volume in thousands of dollars')
p.xaxis.major_label_orientation = np.pi/4

p.vbar(x=label, top=value, width=0.5)

show(p)

In [149]:
# Load the geoenrichment module's country object for the US, and pre-cache some geometry
usa = Country.get("United States")
usa.subgeographies.states['Pennsylvania'].zip5['15275'].geometry

{'rings': [[[-80.20510177197895, 40.46367809284811],
   [-80.20285000000538, 40.464249999514806],
   [-80.2024476590743, 40.461483705836685],
   [-80.19889047299921, 40.45980803625618],
   [-80.1915099992887, 40.458040000417576],
   [-80.19025999991791, 40.458729999882856],
   [-80.1822480065078, 40.45502180658601],
   [-80.17774177478417, 40.45432623157046],
   [-80.17389723916088, 40.45503225869422],
   [-80.17209999967325, 40.45352999976287],
   [-80.1718008488814, 40.45158457909381],
   [-80.17001734741763, 40.45118836250693],
   [-80.16895809843734, 40.4497721307695],
   [-80.17200000037039, 40.449139999580176],
   [-80.1767143003178, 40.44507909347006],
   [-80.18374402816681, 40.44449852306094],
   [-80.18682865784238, 40.44828075360781],
   [-80.18712000042767, 40.45099000006832],
   [-80.1909970157469, 40.450595480067825],
   [-80.19070562375396, 40.44787582669217],
   [-80.19523184940925, 40.45230015926403],
   [-80.19687959659564, 40.452322112927924],
   [-80.19693000049155,

In [150]:
# Create a basic map showing the top five zip codes
top_five_zip_map = gis.map("Pittsburgh")
for zip5 in zip_mean_sales_df.head().index.values.tolist():
    top_five_zip_map.draw(usa.subgeographies.states['Pennsylvania'].zip5[str(zip5)].geometry)
restaurants_sdf.spatial.plot(map_widget=top_five_zip_map, alpha=0.2, cmap='Greys', outline_color='Greys')
top_five_zip_map 

MapView(layout=Layout(height='400px', width='100%'))

###### Section in progress >>> Since we briefly explored zip codes, let's plot a simple choropleth map of zip codes by restaurant sales volume

In [151]:
# def convert_namedarea_dict_to_sedf():
#     return None

In [152]:
# usa.subgeographies.states['Pennsylvania'].counties['Allegheny_County'].zip5

# <span style="color:purple">3. Perform spatial analysis</span> 

To help us identify potential sites for further analysis, we will leverage spatial clustering to find locations with dense retail presence. In this case, Density-based Clustering and Directional Distribution are very useful to help us identify these potential sites. 

## Density-based Clustering

![clustering](https://github.com/Qberto/arcgispythonapi_arcpy-spatial_statistics_examples/raw/5f8994ab80301d8ae145e9081b16c1389a5729d0/img/dbclustering_01.png "Density-based Clustering")

The [Density-based Clustering tool](http://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/how-density-based-clustering-works.htm) works by detecting areas where points are concentrated and where they are separated by areas that are empty or sparse. Points that are not part of a cluster are labeled as noise.

This tool uses unsupervised machine learning clustering algorithms which automatically detect patterns based purely on spatial location and the distance to a specified number of neighbors. These algorithms are considered unsupervised because they do not require any training on what it means to be a cluster.

#### Three clustering algorithms:

The Density-based Clustering tool provides three different Clustering Methods with which to find clusters in your point data:

- _Defined distance (DBSCAN)_ — Uses a specified distance to separate dense clusters from sparser noise. The DBSCAN algorithm is the fastest of the clustering methods, but is only appropriate if there is a very clear Search Distance to use, and that works well for all potential clusters. This requires that all meaningful clusters have similar densities.

- _Self-adjusting (HDBSCAN)_ — Uses a range of distances to separate clusters of varying densities from sparser noise. The HDBSCAN algorithm is the most data-driven of the clustering methods, and thus requires the least user input.

- _Multi-scale (OPTICS)_ — Uses the distance between neighboring features to create a reachability plot which is then used to separate clusters of varying densities from noise. The OPTICS algorithm offers the most flexibility in fine-tuning the clusters that are detected, though it is computationally intensive, particularly with a large Search Distance.

In [153]:
# Path to retail business data
file_geodatabase = "me//unzipped_data//retail.gdb"
businesses_fc = file_geodatabase + "//us_businesses_pittsburgh"
restaurants_fc = file_geodatabase + "//restaurants_pittsburgh"

In [154]:
# Set an output path name
output_path = "{0}//{1}_HDBSCAN".format(file_geodatabase, "RestaurantClusters")
clusters_fc = arcpy.stats.DensityBasedClustering(restaurants_fc, output_path, "HDBSCAN", 5)
clusters_fc

<Result 'me//unzipped_data//retail.gdb/RestaurantClusters_HDBSCAN'>

In [155]:
# Convert the clusters feature class from Density-based Clustering into a spatially-enabeled dataframe
dbclustering_sdf = pd.DataFrame.spatial.from_featureclass(clusters_fc)
clusters_sdf = dbclustering_sdf.loc[dbclustering_sdf['CLUSTER_ID'] != -1]
noise_sdf = dbclustering_sdf.loc[dbclustering_sdf['CLUSTER_ID'] == -1]
clusters_sdf

,OBJECTID,SOURCE_ID,CLUSTER_ID,PROB,OUTLIER,EXEMPLAR,STABILITY,COLOR_ID,SHAPE
3,4,4,56,0.173037,0.826963,0,0.0,7,"{""x"": -8904590.783891981, ""y"": 4868224.0230725..."
4,5,5,36,1.000000,0.000000,1,0.0,4,"{""x"": -8896241.822082484, ""y"": 4844052.5717511..."
5,6,6,36,1.000000,0.000000,1,0.0,4,"{""x"": -8895774.28022115, ""y"": 4844531.07518083..."
6,7,7,36,1.000000,0.000000,1,0.0,4,"{""x"": -8894527.501924265, ""y"": 4845430.1438411..."
7,8,8,36,0.790149,0.209851,0,0.0,4,"{""x"": -8893759.397437794, ""y"": 4846532.3388230..."
8,9,9,7,1.000000,0.000000,1,0.0,8,"{""x"": -8894338.258789917, ""y"": 4828101.1194576..."
9,10,10,56,0.357273,0.642727,0,0.0,7,"{""x"": -8890542.26415387, ""y"": 4867729.84300147..."
10,11,11,35,1.000000,0.000000,1,0.0,3,"{""x"": -8892056.209228657, ""y"": 4860596.0439408..."
11,12,12,7,0.498830,0.501170,0,0.0,8,"{""x"": -8894672.217262294, ""y"": 4827768.1640337..."
13,14,14,36,1.000000,0.000000,1,0.0,4,"{""x"": -8894393.918535315, ""y"": 4844038.0720063..."


In [156]:
noise_sdf

,OBJECTID,SOURCE_ID,CLUSTER_ID,PROB,OUTLIER,EXEMPLAR,STABILITY,COLOR_ID,SHAPE
0,1,1,-1,0.0,0.027113,0,0.0,-1,"{""x"": -8919964.005570533, ""y"": 4867758.9117359..."
1,2,2,-1,0.0,0.023014,0,0.0,-1,"{""x"": -8905258.700836735, ""y"": 4865826.0263644..."
2,3,3,-1,0.0,0.005891,0,0.0,-1,"{""x"": -8905314.36058214, ""y"": 4866727.02381835..."
12,13,13,-1,0.0,0.011129,0,0.0,-1,"{""x"": -8893403.175067253, ""y"": 4859869.8753424..."
26,27,27,-1,0.0,0.255119,0,0.0,-1,"{""x"": -8878608.814740825, ""y"": 4851827.4774533..."
32,33,33,-1,0.0,0.193072,0,0.0,-1,"{""x"": -8879677.48185245, ""y"": 4848287.40122816..."
33,34,34,-1,0.0,0.233182,0,0.0,-1,"{""x"": -8880389.926593523, ""y"": 4855891.4137517..."
34,35,35,-1,0.0,0.153391,0,0.0,-1,"{""x"": -8884475.351905638, ""y"": 4837182.0585866..."
38,39,39,-1,0.0,0.142679,0,0.0,-1,"{""x"": -8878931.641264128, ""y"": 4851725.9003292..."
39,40,40,-1,0.0,0.541517,0,0.0,-1,"{""x"": -8879888.988884956, ""y"": 4853772.1539424..."


In [157]:
results_map = gis.map("Pittsburgh")
clusters_sdf.spatial.plot(map_widget=results_map, renderer_type='u', col="COLOR_ID")
noise_sdf.spatial.plot(map_widget=results_map, cmap='Greys', alpha=0.6, outline_color='Greys')
results_map

/opt/conda/lib/python3.6/site-packages/arcgis/features/geo/_accessor.py:865: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


MapView(layout=Layout(height='400px', width='100%'))

In [158]:
onlyclusters_map = gis.map("Pittsburgh")
clusters_sdf.spatial.plot(map_widget=onlyclusters_map, renderer_type='u', col="COLOR_ID")
onlyclusters_map

MapView(layout=Layout(height='400px', width='100%'))

## Directional Distribution (Standard Deviational Ellipse)

![ellipses](https://github.com/Qberto/arcgispythonapi_arcpy-spatial_statistics_examples/raw/5f8994ab80301d8ae145e9081b16c1389a5729d0/img/ellipses_01.png "Standard Deviational Ellipses") 

While you can get a sense of the clusters by running Density-based Clustering and drawing output cluster features on a map, calculating the standard deviational ellipse makes the trend clear.

In [159]:
# Run Directional Distribution using clusters as an input
clusters_fc = clusters_sdf.spatial.to_featureclass(file_geodatabase+"//restaurant_clusters")
ellipses = arcpy.DirectionalDistribution_stats(clusters_fc, file_geodatabase+"//cluster_ellipses", Case_Field="CLUSTER_ID").getOutput(0)
ellipses_sdf = pd.DataFrame.spatial.from_featureclass(ellipses)
ellipses_sdf

,OBJECTID,CenterX,CenterY,XStdDist,YStdDist,Rotation,CLUSTER_ID,SHAPE
0,1,-8.910530e+06,5.023269e+06,1075.793050,6.262917e+03,22.711797,1,"{""rings"": [[[-8911522.0533, 5023683.927500002]..."
1,2,-8.825619e+06,4.985812e+06,5529.903946,9.611461e+03,23.821812,2,"{""rings"": [[[-8830677.8137, 4988045.345899999]..."
2,3,-8.831966e+06,4.966738e+06,141.501288,2.253131e+03,143.274423,3,"{""rings"": [[[-8832079.3626, 4966653.855099998]..."
3,4,-8.847666e+06,4.849466e+06,1709.881785,6.640863e+03,19.417593,4,"{""rings"": [[[-8849278.3317, 4850034.314499997]..."
4,5,-8.875420e+06,5.007396e+06,1706.386299,6.395992e+03,14.330382,5,"{""rings"": [[[-8877073.3088, 5007818.319300003]..."
5,6,-8.804262e+06,4.922426e+06,9563.355822,3.161194e+03,70.890819,6,"{""rings"": [[[-8795225.4853, 4925556.4595], [-8..."
6,7,-8.894437e+06,4.828135e+06,108.445607,3.014999e+02,24.840462,7,"{""rings"": [[[-8894535.0033, 4828180.4573], [-8..."
7,8,-8.854063e+06,4.977822e+06,2746.045659,5.504024e+02,71.388980,8,"{""rings"": [[[-8851460.4202, 4978698.676899999]..."
8,9,-8.940959e+06,4.907795e+06,1204.757881,2.853235e+03,5.420407,9,"{""rings"": [[[-8942158.2322, 4907908.789899997]..."
9,10,-8.848780e+06,4.984781e+06,101.923075,1.483435e+03,12.375788,10,"{""rings"": [[[-8848879.1986, 4984802.459899999]..."


In [160]:
ellipses_map = gis.map("Pittsburgh")
clusters_sdf.spatial.plot(map_widget=ellipses_map, renderer_type='u', col='CLUSTER_ID', alpha=0.3)
ellipses_sdf.spatial.plot(map_widget=ellipses_map, alpha=0.7, outline_color='Greens')
ellipses_map

MapView(layout=Layout(height='400px', width='100%'))

## Find Hot Spots (Optimized Hot Spots)

To help us identify which clusters contain more successful restaurants, we will use the "Find Hot Spots" tool to identify statistically significant clustering of restaurants using their yearly sales volume as our analysis values. 

In [161]:
# Remove existing layer of restaurant hot spots (if one exists)
gis.content.delete_items(gis.content.search('restaurant_hotspots_01'))

# Run optimized hot spots, using sales volume as the analysis field
restaurant_hot_spots_item = features.analyze_patterns.find_hot_spots(restaurants_item, analysis_field='salesvol', output_name='restaurant_hotspots_01', gis=gis)

In [162]:
hot_spots_map = gis.map("Pittsburgh")
hot_spots_map.add_layer(restaurant_hot_spots_item)

hot_spots_map

MapView(layout=Layout(height='400px', width='100%'))

In [163]:
# Add the cluster ellipses back to the map
ellipses_sdf.spatial.plot(map_widget=hot_spots_map, alpha=0.5)

True

## Where are the top five clustered areas by restaurant sales volume?

In [164]:
# Publish outputs to WebGIS to run summarize tools
ellipses_item = gis.content.import_data(ellipses_sdf)

In [165]:
ellipses_item

<Item title:"f189d12ace794d32b44f6adce7f699cc" type:Feature Layer Collection owner:anieto001>

In [166]:
restaurants_item

<Item title:"Restaurant Points v1" type:Feature Layer Collection owner:anieto001>

In [167]:
from arcgis.features import summarize_data
gis.content.delete_items(gis.content.search("EllipseSummary01"))
sum_fields = ['salesvol Sum', 'salesvol Mean']
sv_summary = summarize_data.aggregate_points(point_layer = restaurants_item.layers[0],
                                             polygon_layer = ellipses_item.layers[0],
                                             summary_fields=sum_fields,
                                             output_name="EllipseSummary01"
                                            )

In [168]:
sv_summary

<Item title:"EllipseSummary01" type:Feature Layer Collection owner:anieto001>

In [169]:
ellipse_summary_map = gis.map("Pittsburgh")
ellipse_summary_map.add_layer(sv_summary)
ellipse_summary_map

MapView(layout=Layout(height='400px', width='100%'))

In [170]:
fset = sv_summary.layers[0].query()
fset

<FeatureSet> 346 features

In [171]:
summary_sdf = fset.sdf
summary_sdf

,SHAPE,analysisarea,centerx,centery,cluster_id,mean_salesvol,objectid,point_count,rotation,sum_salesvol,xstddist,ystddist
0,"{""rings"": [[[-8911522.0533, 5023683.92750001],...",4.637151,-8.910530e+06,5.023269e+06,1,1032.583333,1,12,22.711797,12391.0,1075.793050,6.262917e+03
1,"{""rings"": [[[-8830677.813700002, 4988045.34590...",36.882700,-8.825619e+06,4.985812e+06,2,226.714286,2,7,23.821812,1587.0,5529.903946,9.611461e+03
2,"{""rings"": [[[-8832079.3626, 4966653.855100006]...",0.221052,-8.831966e+06,4.966738e+06,3,248.666667,3,3,143.274423,746.0,141.501288,2.253131e+03
3,"{""rings"": [[[-8849278.331699999, 4850034.31449...",8.097255,-8.847666e+06,4.849466e+06,4,315.000000,4,4,19.417593,1260.0,1709.881785,6.640863e+03
4,"{""rings"": [[[-8877073.308800003, 5007818.31930...",7.538804,-8.875420e+06,5.007396e+06,5,334.666667,5,6,14.330382,2008.0,1706.386299,6.395992e+03
5,"{""rings"": [[[-8795225.485300003, 4925556.45950...",21.247252,-8.804262e+06,4.922426e+06,6,242.800000,6,5,70.890819,1214.0,9563.355822,3.161194e+03
6,"{""rings"": [[[-8894535.003299998, 4828180.4573]...",0.023418,-8.894437e+06,4.828135e+06,7,222.000000,7,4,24.840462,888.0,108.445607,3.014999e+02
7,"{""rings"": [[[-8851460.4202, 4978698.676899999]...",1.050100,-8.854063e+06,4.977822e+06,8,467.200000,8,5,71.388980,2336.0,2746.045659,5.504024e+02
8,"{""rings"": [[[-8942158.2322, 4907908.789900005]...",2.423187,-8.940959e+06,4.907795e+06,9,373.666667,9,3,5.420407,1121.0,1204.757881,2.853235e+03
9,"{""rings"": [[[-8848879.198599996, 4984802.45989...",0.104529,-8.848780e+06,4.984781e+06,10,233.250000,10,4,12.375789,933.0,101.923075,1.483435e+03


In [172]:
top_clusters_sdf = summary_sdf.sort_values(by=['mean_salesvol'], ascending=False)
top_clusters_sdf.head()

,SHAPE,analysisarea,centerx,centery,cluster_id,mean_salesvol,objectid,point_count,rotation,sum_salesvol,xstddist,ystddist
195,"{""rings"": [[[-8923765.116800003, 4931615.50490...",0.061613,-8.923561e+06,4.931611e+06,199,3498.214286,196,14,1.180579,48975.0,204.607454,429.216637
180,"{""rings"": [[[-8906859.582099998, 4949176.86010...",0.501458,-8.907771e+06,4.948683e+06,184,3040.500000,181,4,61.535543,12162.0,1037.134824,691.523068
48,"{""rings"": [[[-8837630.637799999, 4907419.6523]...",0.002831,-8.837612e+06,4.907412e+06,49,3040.333333,49,3,21.222149,9121.0,19.915759,201.982668
235,"{""rings"": [[[-8911277.0898, 4917799.353500001]...",0.011963,-8.911214e+06,4.917822e+06,240,2918.800000,236,5,160.289307,14594.0,66.704668,254.976219
213,"{""rings"": [[[-8880596.077500002, 4919357.89930...",0.027969,-8.881366e+06,4.919723e+06,218,2853.250000,214,4,115.339923,11413.0,852.282767,46.950031


In [173]:
top_clusters_map = gis.map("Pittsburgh")
top_clusters_map.basemap = 'gray'
top_clusters_sdf.head().spatial.plot(map_widget=top_clusters_map)
top_clusters_map

/opt/conda/lib/python3.6/site-packages/arcgis/features/geo/_accessor.py:865: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


MapView(layout=Layout(height='400px', width='100%'))

In [174]:
top_clusters_map2 = gis.map("Pittsburgh")
top_clusters_map2.basemap = 'gray'
top_clusters_map2.add_layer(restaurant_hot_spots_item)
top_clusters_sdf.head().spatial.plot(map_widget=top_clusters_map2)
top_clusters_map2

/opt/conda/lib/python3.6/site-packages/arcgis/features/geo/_accessor.py:865: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


MapView(layout=Layout(height='400px', width='100%'))

In [219]:
# Helper functions to change the extent of a web map based on ellipse centroids
import time

def return_center_dict(centerx, centery):
    center_shape_dict = {}
    center_shape_dict['spatialReference'] = {'latestWkid': 3857, 'wkid': 102100}
    center_shape_dict['x'] = centerx
    center_shape_dict['y'] = centery
    return center_shape_dict

def visit_sites(m, points, zoom_level=15):
    
    m.zoom = zoom_level
    i = 0
    for i in range(len(points)):
        m.center = [xys[i]['y'], xys[i]['x']]
        time.sleep(5)

In [176]:
top_clusters_sdf['center_xy'] = top_clusters_sdf.apply(lambda x: return_center_dict(x['centerx'], x['centery']), axis=1)
top_clusters_sdf.head()

,SHAPE,analysisarea,centerx,centery,cluster_id,mean_salesvol,objectid,point_count,rotation,sum_salesvol,xstddist,ystddist,center_xy
195,"{""rings"": [[[-8923765.116800003, 4931615.50490...",0.061613,-8.923561e+06,4.931611e+06,199,3498.214286,196,14,1.180579,48975.0,204.607454,429.216637,"{'spatialReference': {'latestWkid': 3857, 'wki..."
180,"{""rings"": [[[-8906859.582099998, 4949176.86010...",0.501458,-8.907771e+06,4.948683e+06,184,3040.500000,181,4,61.535543,12162.0,1037.134824,691.523068,"{'spatialReference': {'latestWkid': 3857, 'wki..."
48,"{""rings"": [[[-8837630.637799999, 4907419.6523]...",0.002831,-8.837612e+06,4.907412e+06,49,3040.333333,49,3,21.222149,9121.0,19.915759,201.982668,"{'spatialReference': {'latestWkid': 3857, 'wki..."
235,"{""rings"": [[[-8911277.0898, 4917799.353500001]...",0.011963,-8.911214e+06,4.917822e+06,240,2918.800000,236,5,160.289307,14594.0,66.704668,254.976219,"{'spatialReference': {'latestWkid': 3857, 'wki..."
213,"{""rings"": [[[-8880596.077500002, 4919357.89930...",0.027969,-8.881366e+06,4.919723e+06,218,2853.250000,214,4,115.339923,11413.0,852.282767,46.950031,"{'spatialReference': {'latestWkid': 3857, 'wki..."


In [177]:
# Create list of XY points to visit
from arcgis.geometry import project
sitelist = top_clusters_sdf.head()['center_xy'].tolist()
xys = project(sitelist, in_sr=3857, out_sr=4326)

In [220]:
site_visit_map = gis.map("Pittsburgh")
site_visit_map.basemap = 'gray'
clusters_sdf.spatial.plot(map_widget=site_visit_map, renderer_type='u', col='CLUSTER_ID')
top_clusters_sdf.head().spatial.plot(map_widget=site_visit_map, alpha=0.5)
site_visit_map

/opt/conda/lib/python3.6/site-packages/arcgis/features/geo/_accessor.py:865: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


MapView(layout=Layout(height='400px', width='100%'))

In [221]:
visit_sites(site_visit_map, xys)

# <span style="color:purple">4. Present the Results:</span> 

To present these results, we'll create a Story Map Journal with sections provided by each of our top five ellipsoid zones (determined by average sales volume). 

First, let's save a web map containing the clustered restaurants and top five ellipsoid zones. 

In [222]:
# Create web map properties and use the web map widget's save method
webmap_properties = {'title': 'Pittsburgh Restaurant Zone Prospecting Test01',
                     'snippet': 'Web Map from clustering analysis of restaurants.',
                     'tags': ['automation', 'python']}
site_webmap_item = site_visit_map.save(webmap_properties)
site_webmap_item

<Item title:"Pittsburgh Restaurant Zone Prospecting Test01" type:Web Map owner:anieto001>

The ArcGIS API for Python contains an apps module, with a storymap class. We can leverage this to create the Story Map journal from this hosted notebook.

Let's start by instantiating the story map object, passing it an ID of a story map journal template with no content. 

In [211]:
# Create story map journal with outputs
storymap = arcgis.apps.storymap.JournalStoryMap(item="4b6911a4332a4661bdd89688ade8f04c", gis=gis)
storymap

{"source": "785fc3ee047a45818110133099ff0c8a", "folderId": null, "values": {"settings": {"layout": {"id": "side"}, "appGeocoders": [{"singleLineFieldName": "SingleLine", "name": "ArcGIS World Geocoding Service", "url": "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer"}]}, "title": "Pittsburgh Market Planning: Prioritized Restaurant Zones", "story": {"storage": "WEBAPP", "sections": [{"title": "<span style=\"font-size:40px;\">Pittsburgh Market Planning: Prioritized Restaurant Zones</span>", "content": "<p>This Story Map contains a view of the prioritized zones for further evaluation when considering the opening of a new restaurant in our chain.</p>\n", "contentActions": [], "creaDate": 1540310259660, "pubDate": 1540310259660, "status": "PUBLISHED", "media": {"type": "image", "image": {"url": "https://datascience-premium.esri.com/portal/sharing/rest/content/items/4b6911a4332a4661bdd89688ade8f04c/resources/photo-1537052916033-2916fab31a9e__1540310201229__w1350.jpg", "t

Now we can iterate on each site, store a web map, and add it to a section in the Story Map journal with descriptive content.

In [215]:
storymap_map = gis.map("Pittsburgh")
storymap_map.basemap = 'gray'
clusters_sdf.spatial.plot(map_widget=storymap_map, renderer_type='u', col='CLUSTER_ID')
top_clusters_sdf.head().spatial.plot(map_widget=storymap_map, alpha=0.5)
storymap_map

/opt/conda/lib/python3.6/site-packages/arcgis/features/geo/_accessor.py:865: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._data[col] = self._data[col]


MapView(layout=Layout(height='400px', width='100%'))

In [216]:
# Helper function to change the web map widget extent to each provided ellipse zone
def change_webmap_widget_extent(m, point, zoom_level=15):
    m.zoom = zoom_level
    projected_point = project([point], in_sr=3857, out_sr=4326)
    m.center = [projected_point[0]['y'], projected_point[0]['x']]

In [217]:
storymap_map# Iterate on each of the top five zones for the story map journal sections
counter=0
for index, row in top_clusters_sdf.head().iterrows():
    counter+=1
    print("Adding section {0}...".format(str(counter)))
    
    # Visit site (i.e. set web map widget extent to site)
    change_webmap_widget_extent(storymap_map, row['center_xy'])
    time.sleep(5)
    
    # Store web map on GIS
    webmap_properties = {'title': 'Pittsburgh Restaurant Prospecting: Site {0}'.format(str(counter)),
                         'snippet': 'Web Map from clustering analysis of restaurants.',
                         'tags': ['automation', 'python']}
    site_webmap_item = storymap_map.save(webmap_properties)
    
    section_content = "Average Yearly Sales Volume in Zone: {0}".format(str(round("{:,}".format(row['mean_salesvol']), 2)))
    
    # Add section to storymap object, using the web map ID
    storymap._add_webmap(item= site_webmap_item.id, 
                         title="Pittsburgh Restaurant Prospecting: Site {0}".format(str(counter)),
                         content=section_content,
                         actions=None, visible=True,
                         alt_text="", display='stretch')
    storymap.save(title="AutoStoryMap01", tags="automation", description="auto storymap")
    time.sleep(2)

Adding section 1...
Adding section 2...
Adding section 3...
Adding section 4...
Adding section 5...
